In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline

In [3]:
df = pd.read_excel(r"C:/Users/Janwe/OneDrive/Desktop/Bachelor_Arbeit/Daten/Last_Profile/Profile_VDEW.xls")
df = df.drop(index=[0,98])

In [4]:
df.head()

,Haushalt,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
1,[W],Samstag,Sonntag,Werktag,Samstag,Sonntag,Werktag,Samstag,Sonntag,Werktag
2,00:15:00,70.8,87.5,67.6,89.8,100.1,86.3,80.2,93.4,77.8
3,00:30:00,68.2,81.1,60.8,84.9,92.5,76.9,75.1,86.8,69.6
4,00:45:00,65.9,75,54.9,80.7,85.9,68.8,70.7,81.2,62.4
5,01:00:00,63.3,69.1,49.9,76.6,79.9,62.4,66.6,75.7,56.6


In [6]:
winter = df[['Haushalt', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']]
winter.columns = ["date", "saturday", "sunday", "workday"]
winter = winter.drop(1)
winter.reset_index(inplace=True)
winter = winter.drop(columns="index")

In [7]:
winter.head()

,date,saturday,sunday,workday
0,00:15:00,70.8,87.5,67.6
1,00:30:00,68.2,81.1,60.8
2,00:45:00,65.9,75,54.9
3,01:00:00,63.3,69.1,49.9
4,01:15:00,59.5,63.4,46.2


In [9]:
summer = df[['Haushalt', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']]
summer.columns = ["date", "saturday", "sunday", "workday"]
summer = summer.drop(1)
summer.reset_index(inplace=True,drop=True)

In [10]:
transition = df[['Haushalt', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']]
transition.columns = ["date", "saturday", "sunday", "workday"]
transition = transition.drop(1)
transition.reset_index(inplace=True,drop=True)

In [11]:
def dateTimeList(start,end):
    result=[start]
    while(start < end) :
        start += datetime.timedelta(minutes=15)
        result.append(start)
    return result

In [12]:
h0 = pd.DataFrame(columns=["date","load"])
start = datetime.datetime(2019,4,1,0,15,0)
end = datetime.datetime(2013,4,8,0,0,0)

h0["date"] = dateTimeList(start,end)

res = -1
ind =[]

for i in range(len(h0)):
    if(res == 95):
        res=-1
    res += 1
    ind.append(res)
    
h0["Id"] = ind
summer.index.name = "Id"
h0_final = h0.merge(summer, how='left',on="Id")

In [13]:
def specLoad(row):
    if(row["date_x"].dayofweek<5):
        if(row["date_x"].dayofweek==0 and row["date_y"] == "00:00:00"):
            return row["sunday"]
        return row["workday"]
    elif(row["date_x"].dayofweek==5):
        if(row["date_y"] == "00:00:00"):
            return row["workday"]
        return row["saturday"]
    elif(row["date_x"].dayofweek==6):
        if(row["date_y"] == "00:00:00"):
            return row["saturday"]
        return row["sunday"]

In [14]:
h0_final["load"] = h0_final.apply(lambda row: specLoad(row),axis=1)
h0_final.rename(columns={"date_x" : "date"},inplace=True)
h0_final = h0_final[["date","load"]]

In [15]:
h0_summer_date = h0_final.copy()

In [16]:
#converte Wh to kWh
h0_summer_date["load"] = h0_summer_date["load"]/1000
#converte kWh to MWh
h0_summer_date["load"] = h0_summer_date["load"]/1000

In [21]:
#Multiply with the number of households
h0_summer_date["load"] = h0_summer_date["load"]*65000

In [35]:
#Dynamisierungsfaktor F

def dynamismFactor(t):
    return (-3.92*10**-10)*t**4 + (3.2*10**-7)*t**3 - (7.02*10**-5)*t**2 + (2.1*10**-3)*t + 1.24